# Load Packages

In [ ]:

%load_ext autoreload
%autoreload 2

import sys
from os.path import join
from tqdm.auto import tqdm
import joblib
import torch
sys.path.append("../../")

from src.file_manager.load_data import load_split_dict
from src.file_processing.processing_predictions import load_prediction_df_dict, save_pi_df_dict
from src.file_manager.save_load_scaler import load_scaler
from src.pi_methods.knn import knn_prediction_interval
from src.pi_methods.weighted import weighted_prediction_interval
from src.pi_methods.cond_gaussian import cond_gauss_prediction_interval
from src.pi_methods.gaussian_copula import gauss_copula_prediction_interval
from src.pi_methods.conformal_prediction import conformal_prediction_interval
from src.misc import create_folder
from ue_pi_dicts import ue_dict
from seed_file import seed

# seed = 2023
data_label = "physionet"
batch_size = 64

# File paths
fp_notebooks_folder = "../"
fp_project_folder = join(fp_notebooks_folder, "../")
fp_processed_data_folder = join(fp_project_folder, "processed_data")
fp_output_data_folder = join(fp_processed_data_folder, "physionet")
fp_checkpoint_folder = join(fp_project_folder, "checkpoints")
fp_project_checkpoints = join(fp_checkpoint_folder, data_label)
fp_tuning = join(fp_project_checkpoints, "tuning")
fp_models = join(fp_project_checkpoints, "models")
fp_predictions = join(fp_project_checkpoints, "predictions")
fp_pi_predictions = join(fp_project_checkpoints, "pi_predictions")

# Seed filepaths
fp_cur_model_folder = join(fp_models, str(seed))
create_folder(fp_cur_model_folder)
fp_cur_predictions_folder = join(fp_predictions, str(seed))
create_folder(fp_cur_predictions_folder)
fp_cur_pi_predictions_folder = join(fp_pi_predictions, str(seed))
create_folder(fp_cur_pi_predictions_folder)

pred_file_names =  ["rue", "gpr", "infernoise", "der", "bnn"]

# Load Data

In [ ]:
split_dict = load_split_dict(fp_output_data_folder)

# Load All Predictions

In [ ]:
scaler = load_scaler(join(fp_output_data_folder, "scaler.pickle"))
pred_df_dict = load_prediction_df_dict(
    split_dict, fp_cur_predictions_folder, pred_file_names=pred_file_names) 

# Add KNN PI

In [ ]:
# Apply knn_prediction_interval to all RUE
ue_label = "RUE"
for time_label, time_info in tqdm(pred_df_dict.items()):
    val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
    pred_label, ue_col = ue_dict[ue_label]["pred_label"], ue_dict[ue_label]["ue_col"]
    pred_df_dict[time_label]["test_df"] = knn_prediction_interval(
        df_val=val_df, df_test=test_df, 
        predictors=split_dict["feat_cols"], pred_cols=pred_cols, 
        pred_label=pred_label, regressor_label=time_label, ue_col=ue_col, 
        scaler=scaler, seed=seed
    )


# Add Weighted Percentile PI

In [ ]:
# Apply weighted_prediction_interval to all RUE
ue_label = "RUE"
for time_label, time_info in tqdm(pred_df_dict.items()):
    val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
    pred_label, ue_col = ue_dict[ue_label]["pred_label"], ue_dict[ue_label]["ue_col"]
    pred_df_dict[time_label]["test_df"] = weighted_prediction_interval(
        df_val=val_df, df_test=test_df, 
        predictors=split_dict["feat_cols"], pred_cols=pred_cols, 
        pred_label=pred_label, regressor_label=time_label, ue_col=ue_col, 
        scaler=scaler, seed=seed
    )

# Add Cond Gauss PI

In [ ]:
# Apply cond_gauss_prediction_interval to all RUE
ue_label = "RUE"
for time_label, time_info in tqdm(pred_df_dict.items()):
    val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
    pred_label, ue_col = ue_dict[ue_label]["pred_label"], ue_dict[ue_label]["ue_col"]
    pred_df_dict[time_label]["test_df"] = cond_gauss_prediction_interval(
        df_val=val_df, df_test=test_df, predictors=split_dict["feat_cols"], pred_cols=pred_cols, 
        pred_label=pred_label, regressor_label=time_label, ue_col=ue_col, 
        scaler=scaler, seed=seed
    )

# Add Gaussian Copula PI

In [ ]:
ue_label = "RUE"
for time_label, time_info in tqdm(pred_df_dict.items()):
    val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
    pred_label, ue_col = ue_dict[ue_label]["pred_label"], ue_dict[ue_label]["ue_col"]
    pred_df_dict[time_label]["test_df"] = gauss_copula_prediction_interval(
        df_val=val_df, df_test=test_df, 
        predictors=split_dict["feat_cols"], pred_cols=pred_cols, 
        pred_label=pred_label, regressor_label=time_label, ue_col=ue_col, 
        scaler=scaler, seed=seed
    )

# Add CP

In [ ]:
# Apply conformal prediction to all UEs
for time_label, time_info in tqdm(pred_df_dict.items()):
    val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
    for ue_label, ue_info in ue_dict.items():
        val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
        pred_label, ue_col = ue_info["pred_label"], ue_info["ue_col"]
        pred_df_dict[time_label]["test_df"] = conformal_prediction_interval(
            df_val=val_df, df_test=test_df, 
            predictors=split_dict["feat_cols"], pred_cols=pred_cols, 
            pred_label=pred_label, regressor_label=time_label, ue_col=ue_col, 
            scaler=scaler, seed=seed
        )

# Export df dict

In [ ]:
save_pi_df_dict(pred_df_dict, fp_cur_pi_predictions_folder)